In [48]:
import requests
from bs4 import BeautifulSoup
import time
import sys
import pypatent
import gevent

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}

#### AN/NikonAND PPPD/1/1/2017->10/27/2020   (470)
http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=1&f=S&l=50&Query=AN%2FNikon+AND+PPPD%2F1%2F1%2F2017-%3E10%2F27%2F2020&d=PTXT
#### AN/OPTOTECH OPTIKMASCHINEN GMBH AND PPPD/1/1/2017->10/27/2020   (13)
http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&f=S&l=50&d=PTXT&RS=%28AN%2FOptikmaschinen+AND+PPPD%2F19920101-%3E20201027%29&Refine=Refine+Search&Query=AN%2FOPTIKMASCHINEN+AND+PPPD%2F1%2F1%2F1992-%3E10%2F27%2F2020
#### AN/CARL ZEISS AND PPPD/1/1/2017->10/27/2020

#### AN/Satisloh AND PPPD/1/1/1992->10/27/2020   (49)
http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&f=S&l=50&d=PTXT&RS=%28AN%2FNikon+AND+PPPD%2F20170101-%3E20201027%29&Refine=Refine+Search&Query=AN%2FSatisloh+AND+PPPD%2F1%2F1%2F1992-%3E10%2F27%2F2020
#### AN/ESSILOR AND PPPD/1/1/2017->10/27/2020   (146)
http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&f=S&l=50&d=PTXT&RS=%28AN%2FSatisloh+AND+PPPD%2F19920101-%3E20201027%29&Refine=Refine+Search&Query=AN%2FESSILOR+AND+PPPD%2F1%2F1%2F2017-%3E10%2F27%2F2020
#### AN/Schneider GmbH AND PPPD/1/1/2017->10/27/2020

#### AN/QED Technologies International AND PPPD/1/1/1992->10/27/2020

#### AN/Trioptics GmbH AND PPPD/1/1/1992->10/27/2020


In [49]:
url = 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&f=S&l=50&d=PTXT&RS=%28AN%2FOptikmaschinen+AND+PPPD%2F19920101-%3E20201027%29&Refine=Refine+Search&Query=AN%2FOPTIKMASCHINEN+AND+PPPD%2F1%2F1%2F1992-%3E10%2F27%2F2020'
page = 1
company = "OPTOTECH"

In [14]:
res = requests.get(url, headers=headers) 
res.encoding = 'utf8'
content = res.text
soup = BeautifulSoup(content, "html.parser")
posts = soup.find_all(valign = "top")

flag = 0
fp = open("link_" + company + ".txt", "w")
print(url)
for i in range(page):
    for post in posts:
        if flag==2:
    #         print(post)
    #         print("----------------")
            p = post.find("a")
            url_link = p.get('href')
            url_link = "http://patft.uspto.gov" + url_link
            in_str = url_link + '\n'
#             print(in_str)
            
            fp.writelines(in_str)
#             print(p.string)
#             print(url_link)
#             print("=================================")
            flag=0
        else:
            flag+=1

    if page > 1:
        center = soup.center
        center_a = center.find_all("a")
        if i == 0:
            next_url = "http://patft.uspto.gov" + center_a[6].get("href")
        else:
            next_url = "http://patft.uspto.gov" + center_a[7].get("href")
        print(i+1)
        print('--------')
        print(next_url)
        res = requests.get(next_url, headers=headers) 
        res.encoding = 'utf8'
        content = res.text
        soup = BeautifulSoup(content, "html.parser")
        posts = soup.find_all(valign = "top")
        time.sleep(0.2)
fp.close()

In [86]:
f = open("link_" + company + ".txt", "r")
f_data = open("data_" + company + ".csv", "w")
lines = f.readlines()
f.close()
start = 0
start_lines = lines[start:]
print("toatl: " + str(len(lines)) + "  | start from: " + str(start) + "  | last: " + str(len(start_lines)))


for i,line in enumerate(start_lines):
    url = line
    url = url.strip('\n')
    # print(line)
    res = requests.get(url, headers=headers)
    content = res.text
    soup = BeautifulSoup(content, "html.parser")
    res.encoding = 'utf8'
    time.sleep(0.2)
    
    tables = soup.find_all('table')
    new_txt = ""
    ths = soup.find_all('tr')
    for th in ths:
        t = th.get_text('')
        if "Assignee" in t:
            txt = t[t.find("Assignee"):]
            new_txt = txt.replace('\n','')
            new_txt = new_txt[:9] + '\t' + new_txt[9:]
#             print(new_txt)
            print("fuck tr--------------")
            break
    
    for table in tables:
        t = table.get_text('')

        if "Assignee" in t:
            txt = t[t.find("Assignee"):t.find("Family")]
            new_txt = txt.replace('\n','')
            new_txt = new_txt[:9] + '\t' + new_txt[9:]
#             print(new_txt)
            print("fuck table--------------")
            
        if "Document Identifier" in t:
            txt = table.get_text('\t', strip=True )
            new_txt += ('\t' + txt + '\t')
        if "Current" in t:
            txt = table.get_text('', strip=True )
            txt = txt.replace('Current', '\tCurrent')
            txt = txt.replace('Class:', 'Class:\t')
            if "Field" in t:
                txt = txt[:txt.find("Field")]
            new_txt += (txt + '\n')
            break
            
    print(i + start)
    print(new_txt)
    print("=================")
    f_data.writelines(new_txt)

f_data.close()

toatl: 13  | start from: 0  | last: 13
fuck tr--------------
0
Assignee:	OptoTech Optikmaschinen GmbH (Wettenberg, DE)	Document Identifier	Publication Date	US 20160250694 A1	Sep 1, 2016		Current U.S. Class:	1/1	Current CPC Class:	B23Q 7/1421 (20130101); B23C 3/16 (20130101); B23Q 39/04 (20130101); B23Q 7/048 (20130101); B23B 9/00 (20130101); B23C 3/00 (20130101); B23B 3/065 (20130101); B23C 2220/04 (20130101); B23Q 11/08 (20130101); Y10T 409/307784 (20150115); Y10T 409/308344 (20150115); B23C 2270/14 (20130101); Y10T 409/3056 (20150115); Y10T 409/305656 (20150115); B23Q 2039/008 (20130101); Y10T 29/5109 (20150115); B23C 2215/40 (20130101); Y10T 409/30756 (20150115); B23C 2226/61 (20130101); B23Q 2039/002 (20130101)	Current International Class:	B23C 3/00 (20060101); B23Q 39/04 (20060101); B23Q 7/04 (20060101); B23B 3/06 (20060101); B23C 3/16 (20060101); B23B 9/00 (20060101); B23B 9/10 (20060101); B23Q 7/14 (20060101); B23Q 11/08 (20060101); B23Q 39/00 (20060101)

fuck tr--------------
1

In [5]:
print("==============DONE==============")

==============DONE==============


In [84]:
# # url = 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=13&f=G&l=50&d=PTXT&p=1&S1=(OPTIKMASCHINEN.ASNM.+AND+%40PPPD%3E%3D19920101%3C%3D20201027)&OS=AN/OPTIKMASCHINEN+AND+PPPD/1/1/1992-%3E10/27/2020&RS=(AN/OPTIKMASCHINEN+AND+PPPD/19920101-%3E20201027)'
# # url = 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=12&f=G&l=50&d=PTXT&p=1&S1=(OPTIKMASCHINEN.ASNM.+AND+%40PPPD%3E%3D19920101%3C%3D20201027)&OS=AN/OPTIKMASCHINEN+AND+PPPD/1/1/1992-%3E10/27/2020&RS=(AN/OPTIKMASCHINEN+AND+PPPD/19920101-%3E20201027)'
# # url = 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=1&f=G&l=50&d=PTXT&p=1&S1=(OPTIKMASCHINEN.ASNM.+AND+%40PPPD%3E%3D19920101%3C%3D20201027)&OS=AN/OPTIKMASCHINEN+AND+PPPD/1/1/1992-%3E10/27/2020&RS=(AN/OPTIKMASCHINEN+AND+PPPD/19920101-%3E20201027)'
# url = 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=13&f=G&l=50&d=PTXT&p=1&S1=(OPTIKMASCHINEN.ASNM.+AND+%40PPPD%3E%3D19920101%3C%3D20201027)&OS=AN/OPTIKMASCHINEN+AND+PPPD/1/1/1992->10/27/2020&RS=(AN/OPTIKMASCHINEN+AND+PPPD/19920101->20201027)'
# # print(line)
# res = requests.get(url, headers=headers)
# content = res.text
# soup = BeautifulSoup(content, "html.parser")
# res.encoding = 'utf8'

# tables = soup.find_all('table')
# new_txt = ""

# # print(soup.find_all('th'))
# ths = soup.find_all('tr')
# for th in ths:
#     t = th.get_text('')
#     if "Assignee" in t:
#         txt = t[t.find("Assignee"):]
#         new_txt = txt.replace('\n','')
#         new_txt = new_txt[:9] + '\t' + new_txt[9:]
# #         print(new_txt)
# #         print("fuck tr--------------")
#         break
        
# for i,table in enumerate(tables):
#     t = table.get_text('')
#     if "Assignee" in t:
#         txt = table.get_text('')
#         txt = txt[txt.find("Assignee"):txt.find("Family")]
#         new_txt = txt.replace('\n','')
#         new_txt = new_txt[:9] + '\t' + new_txt[9:]
# #         print("Assignee_" + str(i) + ": ")
# #         print(new_txt)
# #         print('-------------------')
#     if "Document Identifier" in t:
#         txt = table.get_text('\t', strip=True )
#         new_txt += ('\t' + txt)
# #         print("Document Identifier_" + str(i) + ": ")
# #         print(new_txt)
# #         print('-------------------')
#     if "Current" in t:
#         txt = table.get_text('', strip=True )
#         txt = txt.replace('Current', '\tCurrent')
#         txt = txt.replace('Class:', 'Class:\t')
#         if "Field" in t:
#             txt = txt[:txt.find("Field")]
        
#         new_txt += (txt + '\t')
#         print("Current_" + str(i) + ": ")
#         print(new_txt)
#         print('-------------------')
#         break
        

# # print(new_txt)

Current_6: 
Assignee:	Loh Optikmaschinen AG (Wetzler, DE)	Document Identifier	Publication Date	US 20030043343 A1	Mar 6, 2003	Current U.S. Class:	451/65; 407/53; 451/461	Current CPC Class:	B24B 13/01 (20130101); B23C 3/023 (20130101); B24B 13/043 (20130101); B23C 5/2273 (20130101); B29D 11/00932 (20130101); B23C 5/006 (20130101); B24B 13/06 (20130101); B23B 2226/31 (20130101); B23C 2226/61 (20130101); Y10T 407/1946 (20150115); B23C 2200/203 (20130101); B23C 2226/315 (20130101)	Current International Class:	B24B 13/00 (20060101)	
-------------------
